In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("JoinApp").getOrCreate()
sc=spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/22 10:50:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
customerDf = spark.read.option("sep","\t").option("header",True).option("inferSchema",True).csv("/home/labuser/Downloads/Sales/customer.csv")
lineitemDf = spark.read.option("sep","\t").option("header",True).option("inferSchema",True).csv("/home/labuser/Downloads/Sales/lineitem.csv")
nationDf = spark.read.option("sep","\t").option("header",True).option("inferSchema",True).csv("/home/labuser/Downloads/Sales/nation.csv")
ordersDf = spark.read.option("sep","\t").option("header",True).option("inferSchema",True).csv("/home/labuser/Downloads/Sales/orders.csv")
partDf = spark.read.option("sep","\t").option("header",True).option("inferSchema",True).csv("/home/labuser/Downloads/Sales/part.csv")
partsuppDf = spark.read.option("sep","\t").option("header",True).option("inferSchema",True).csv("/home/labuser/Downloads/Sales/partsupp.csv")
regionDf = spark.read.option("sep","\t").option("header",True).option("inferSchema",True).csv("/home/labuser/Downloads/Sales/region.csv")
supplierDf = spark.read.option("sep","\t").option("header",True).option("inferSchema",True).csv("/home/labuser/Downloads/Sales/supplier.csv")

In [3]:
customerDf.printSchema()

root
 |-- C_CUSTKEY: integer (nullable = true)
 |-- C_NAME: string (nullable = true)
 |-- C_ADDRESS: string (nullable = true)
 |-- C_NATIONKEY: integer (nullable = true)
 |-- C_PHONE: string (nullable = true)
 |-- C_ACCTBAL: double (nullable = true)
 |-- C_MKTSEGMENT: string (nullable = true)
 |-- C_COMMENT: string (nullable = true)



In [4]:
lineitemDf.printSchema()

root
 |-- L_ORDERKEY: integer (nullable = true)
 |-- L_PARTKEY: integer (nullable = true)
 |-- L_SUPPKEY: integer (nullable = true)
 |-- L_LINENUMBER: integer (nullable = true)
 |-- L_QUANTITY: double (nullable = true)
 |-- L_EXTENDEDPRICE: double (nullable = true)
 |-- L_DISCOUNT: double (nullable = true)
 |-- L_TAX: double (nullable = true)
 |-- L_RETURNFLAG: string (nullable = true)
 |-- L_LINESTATUS: string (nullable = true)
 |-- L_SHIPDATE: date (nullable = true)
 |-- L_COMMITDATE: date (nullable = true)
 |-- L_RECEIPTDATE: date (nullable = true)
 |-- L_SHIPINSTRUCT: string (nullable = true)
 |-- L_SHIPMODE: string (nullable = true)
 |-- L_COMMENT: string (nullable = true)



In [5]:
nationDf.printSchema()

root
 |-- N_NATIONKEY: integer (nullable = true)
 |-- N_NAME: string (nullable = true)
 |-- N_REGIONKEY: integer (nullable = true)
 |-- N_COMMENT: string (nullable = true)



In [6]:
ordersDf.printSchema()

root
 |-- O_ORDERKEY: integer (nullable = true)
 |-- O_CUSTKEY: integer (nullable = true)
 |-- O_ORDERSTATUS: string (nullable = true)
 |-- O_TOTALPRICE: double (nullable = true)
 |-- O_ORDERDATE: date (nullable = true)
 |-- O_ORDERPRIORITY: string (nullable = true)
 |-- O_CLERK: string (nullable = true)
 |-- O_SHIPPRIORITY: integer (nullable = true)
 |-- O_COMMENT: string (nullable = true)



In [7]:
cusInfoDf = nationDf.join(customerDf,nationDf.N_NATIONKEY == customerDf.C_NATIONKEY).join(ordersDf, customerDf.C_CUSTKEY == ordersDf.O_CUSTKEY).join(lineitemDf,ordersDf.O_ORDERKEY == lineitemDf.L_ORDERKEY).select(customerDf["*"],nationDf["N_NAME"].alias("Nation"),ordersDf["O_ORDERKEY"].alias("Order_Key"),ordersDf["O_TOTALPRICE"].alias("Total_Price"),lineitemDf["L_LINENUMBER"].alias("Line_Number"))
cusInfoDf.show(3)

+---------+------------------+-----------------+-----------+---------------+---------+------------+--------------------+-------+---------+-----------+-----------+
|C_CUSTKEY|            C_NAME|        C_ADDRESS|C_NATIONKEY|        C_PHONE|C_ACCTBAL|C_MKTSEGMENT|           C_COMMENT| Nation|Order_Key|Total_Price|Line_Number|
+---------+------------------+-----------------+-----------+---------------+---------+------------+--------------------+-------+---------+-----------+-----------+
|        1|Customer#000000001|IVhzIApeRb ot,c,E|         15|25-989-741-2988|   711.56|    BUILDING|to the even, regu...|MOROCCO|    53283|  165928.33|          4|
|        1|Customer#000000001|IVhzIApeRb ot,c,E|         15|25-989-741-2988|   711.56|    BUILDING|to the even, regu...|MOROCCO|    53283|  165928.33|          3|
|        1|Customer#000000001|IVhzIApeRb ot,c,E|         15|25-989-741-2988|   711.56|    BUILDING|to the even, regu...|MOROCCO|    53283|  165928.33|          2|
+---------+-----------

In [8]:
cusInfoDf.printSchema()

root
 |-- C_CUSTKEY: integer (nullable = true)
 |-- C_NAME: string (nullable = true)
 |-- C_ADDRESS: string (nullable = true)
 |-- C_NATIONKEY: integer (nullable = true)
 |-- C_PHONE: string (nullable = true)
 |-- C_ACCTBAL: double (nullable = true)
 |-- C_MKTSEGMENT: string (nullable = true)
 |-- C_COMMENT: string (nullable = true)
 |-- Nation: string (nullable = true)
 |-- Order_Key: integer (nullable = true)
 |-- Total_Price: double (nullable = true)
 |-- Line_Number: integer (nullable = true)



In [9]:
cusInfoDf.show()

+---------+------------------+-----------------+-----------+---------------+---------+------------+--------------------+-------+---------+-----------+-----------+
|C_CUSTKEY|            C_NAME|        C_ADDRESS|C_NATIONKEY|        C_PHONE|C_ACCTBAL|C_MKTSEGMENT|           C_COMMENT| Nation|Order_Key|Total_Price|Line_Number|
+---------+------------------+-----------------+-----------+---------------+---------+------------+--------------------+-------+---------+-----------+-----------+
|        1|Customer#000000001|IVhzIApeRb ot,c,E|         15|25-989-741-2988|   711.56|    BUILDING|to the even, regu...|MOROCCO|    53283|  165928.33|          4|
|        1|Customer#000000001|IVhzIApeRb ot,c,E|         15|25-989-741-2988|   711.56|    BUILDING|to the even, regu...|MOROCCO|    53283|  165928.33|          3|
|        1|Customer#000000001|IVhzIApeRb ot,c,E|         15|25-989-741-2988|   711.56|    BUILDING|to the even, regu...|MOROCCO|    53283|  165928.33|          2|
|        1|Customer#00

In [10]:
cusInfoDf = cusInfoDf.withColumnRenamed("C_CUSTKEY","Customer_Key").withColumnRenamed("C_NAME","Customer_Name").withColumnRenamed("C_ADDRESS","Customer_Address").withColumnRenamed("C_PHONE","Customer_Phone")

In [12]:
from pyspark.sql.functions import *

In [13]:
cusInfoDf = cusInfoDf.drop(col("C_NATIONKEY"),col("C_ACCTBAL"),col("C_MKTSEGMENT"),col("C_COMMENT"))

In [14]:
totalSpendDf = cusInfoDf.groupBy("Customer_Key").sum("Total_Price")
totalSpendDf.show()

+------------+--------------------+
|Customer_Key|    sum(Total_Price)|
+------------+--------------------+
|         148|   9389344.430000003|
|         463|   9400948.980000002|
|         496|1.6688017019999996E7|
|         833|   8526686.990000004|
|        1088|  4240906.7700000005|
|        1238|   8150999.600000001|
|        1342|1.3610552759999994E7|
|         392|   6364371.840000003|
|         623|   9931784.389999999|
|         737|1.5131369519999987E7|
|        1025|   6676951.369999998|
|        1084|1.3664868309999987E7|
|        1127|  4099256.4500000016|
|        1460|   6807579.039999998|
|        1483|1.4234627099999998E7|
|          31|   9781158.139999997|
|        1139|  3315328.9999999995|
|        1270|1.5178320019999996E7|
|        1303|1.2708241649999999E7|
|        1322|   5575107.279999998|
+------------+--------------------+
only showing top 20 rows



In [15]:
cusInfoDf = cusInfoDf.withColumn("Customer_Phone",regexp_replace("Customer_Phone","-",""))
cusInfoDf.show(3)

+------------+------------------+-----------------+--------------+-------+---------+-----------+-----------+
|Customer_Key|     Customer_Name| Customer_Address|Customer_Phone| Nation|Order_Key|Total_Price|Line_Number|
+------------+------------------+-----------------+--------------+-------+---------+-----------+-----------+
|           1|Customer#000000001|IVhzIApeRb ot,c,E|  259897412988|MOROCCO|    53283|  165928.33|          4|
|           1|Customer#000000001|IVhzIApeRb ot,c,E|  259897412988|MOROCCO|    53283|  165928.33|          3|
|           1|Customer#000000001|IVhzIApeRb ot,c,E|  259897412988|MOROCCO|    53283|  165928.33|          2|
+------------+------------------+-----------------+--------------+-------+---------+-----------+-----------+
only showing top 3 rows



In [ ]:
cusInfoDf.show(4)

In [16]:
cusInfoDf.write.mode("overwrite").csv("/home/labuser/Downloads/Sales/Output")

In [17]:
cusInfoDf.write.mode("overwrite").parquet("/home/labuser/Downloads/Sales/Pout")